In [ ]:
using Pkg
Pkg.activate("../EEG_plots/env")

In [ ]:
using Makie
using GLMakie
using PyMNE
using JLD2 # loading data
using TopoPlots
using StatsBase # mean/std
using Pipe
using ColorSchemes
using Colors
using LinearAlgebra

# 5. Topoplot array

# 5.1

In [ ]:
# important functions for the main plot
function del_empt2(coord)
    # searching for unsued coordinates

    tmp = Array(minimum(coord):maximum(coord))
    unused = []
    for i in tmp
        if !(i in coord)
            append!(unused, i)
        end
    end
    
    # creation of new coordinates without empty ones
    newcoord = []
    for i in coord
        q = 0
        for j in unused  
            if i > j
                q = findall(x->x==j, unused)[1]
            end
        end
        append!(newcoord, i - q)
    end 
    newcoord
end

function del_empt(pos)
    x = [i[1] for i in pos]
    y = [i[2] for i in pos]
    
    x = round.(x; digits=2)
    y = Integer.(round.((y .- mean(y))  * 20)) * -1
    x = Integer.(round.((x .- mean(x))  * 20))

    newx = del_empt2(x)
    newy = del_empt2(y)

    # twiching the lateral electodes' positons
    ma = findall(xx->xx==maximum(newx), newx)[1]
    mi = findall(xx->xx==minimum(newx), newx)[1]
    newx[mi] = newx[mi] + 1
    newx[ma] = newx[ma] - 1
    # return
    newx, newy
end

In [ ]:
# the main plot
let 
    data = @pipe results_onesubject |> 
        @subset(_, :channel .< 31 ) |>
        @subset(_, :coefname .== ("(Intercept)")) |>
        select(_, Not([:basisname, :group, :subject, :coefname])) |>
        groupby(_, :channel) |> 
        transform(_, [:estimate,:time] => (x,t) -> x .- mean(x[t .< 0])) |> # baseline correction using lambda function 
        rename!(_, :estimate_time_function => :estimate_bsln_corrected)

    f = Figure()
    gl = f[1, 1] = GridLayout()
    x, y = del_empt(pos)

    for i in 1:30
        col = x[i]
        row = y[i]

        if (row == -1 && col == 0)
            col = 1
        end
        if (row == -8 && col == 1)
            col = 2
        end
        ax = Axis(gl[row, col], title = raw.ch_names[i], #string(row, ' ',  col),
        titlealign = :right, titlesize = 12) # attribte for position 
        #println(raw.ch_names[i], " ",  row, " ", col)
        hidespines!(ax) 
        hidedecorations!(ax)
        tmp = data[data[:, :channel] .== i, :] 
        lines!(tmp.time, tmp.estimate_bsln_corrected)
        hlines!(0, color = :gray, linewidth = 1)
        vlines!(0, color = :gray, linewidth = 1)
        Makie.ylims!(low = -10, high = 14)
        #Makie.limits!(ax, BBox(-2, 2, -2, 2))
    end
    Makie.colgap!(gl, 5)
    Makie.rowgap!(gl, 5)
    f
    #save("plots/plot5.1.svg", f)
end

# 5.2

In [ ]:
begin
    data = @pipe results_onesubject |> 
        @subset(_, :channel .< 31 ) |>
        @subset(_, :coefname .== ("(Intercept)")) |>
        select(_, Not([:basisname, :group, :subject, :coefname])) |>
        groupby(_, :channel) |> 
        transform(_, [:estimate,:time] => (x,t) -> x .- mean(x[t .< 0])) |> # baseline correction using lambda function 
        rename!(_, :estimate_time_function => :estimate_bsln_corrected)
        
    
    fig = Figure()
    ax = Axis(fig[1, 1])

    i = 1
    Makie.xlims!(low = 20, high = 170)
    Makie.ylims!(low = 30, high = 200)
    
    xs = 0:0.01:10
    ys = 0.5 .* sin.(xs)

    xscale = 8
    yscale = 0.5
    
    for i in 1:30
        xmove = pos[i][1] * 200
        ymove =  pos[i][2] * 200 
        #println(raw.ch_names[i], " ", xmove, " ", ymove, " ")
        m4x4 = Makie.Mat4f(xscale, 0, 0, 0,
                        0, yscale, 0, 0,
                        0, 0, 1, 0,
                        xmove, ymove , 0, 1)
        tmp = data[data[:, :channel] .== i, :] 
        lines!(tmp.time, tmp.estimate_bsln_corrected, model = m4x4, color = "#0077b6")
        text!(xmove, ymove + 6, text = raw.ch_names[i], 
        align = (:left, :bottom), color = :black,
        textsize = 12)
        #hlines!(xmove, color = :black, linewidth = 1, model = m4x4)
        vlines!(0,  xmin = minimum(tmp.estimate_bsln_corrected),
        xmax = maximum(tmp.estimate_bsln_corrected),  color = :red, linewidth = 1)
    end

    hidespines!(ax) 
    hidedecorations!(ax)
    fig
    #save("plots/plot5.svg", fig)
end

begin
    data = @pipe results_onesubject |> 
        @subset(_, :channel .< 31 ) |>
        @subset(_, :coefname .== ("(Intercept)")) |>
        select(_, Not([:basisname, :group, :subject, :coefname])) |>
        groupby(_, :channel) |> 
        transform(_, [:estimate,:time] => (x,t) -> x .- mean(x[t .< 0])) |> # baseline correction using lambda function 
        rename!(_, :estimate_time_function => :estimate_bsln_corrected)
        
    
    fig = Figure()
    ax = Axis(fig[1, 1])

    Makie.xlims!(low = 20, high = 170)
    Makie.ylims!(low = 30, high = 200)
    
    xscale = 8
    yscale = 0.5
    
    for i in 1:30
        xmove = pos[i][1] * 200
        ymove =  pos[i][2] * 200 
        #println(raw.ch_names[i], " ", xmove, " ", ymove, " ")
        m4x4 = Makie.Mat4f(xscale, 0, 0, 0,
                        0, yscale, 0, 0,
                        0, 0, 1, 0,
                        xmove, ymove , 0, 1)
        tmp = data[data[:, :channel] .== i, :]
        a = pos[i][1] 
        a = (a + (a - 0.4660183)* 0.35)  
        b = a + 0.09 
        hlines!(ymove, xmin = a, xmax = b, 
         color = :black, linewidth = 1)

        a = pos[i][2] 
        a = a - (1 - a)/3.2* 0.7 
        #a = a - (0.96 - a)/3.2* 0.7
        b = a + 0.04 
        vlines!(xmove, ymin = a, ymax = b, color = :black, linewidth = 1)

        lines!(tmp.time, tmp.estimate_bsln_corrected, model = m4x4, color = "#0077b6")
        text!(xmove, ymove + 6, text = raw.ch_names[i], 
        align = (:left, :bottom), color = :black,
        textsize = 12)
    
        #println(raw.ch_names[i], " ", pos[i][2] , " ",  a )
    end

    hidespines!(ax) 
    hidedecorations!(ax)
    fig
    #save("plots/plot5.svg", fig)
end